In [ ]:
# old scripts
def unprotect_xlsx(filename):
    import win32com.client
    xcl = win32com.client.Dispatch('Excel.Application')
    pw_str = '12345'
    wb = xcl.workbooks.open(filename)
    wb.Unprotect(pw_str)
    wb.UnprotectSharing(pw_str)
    xcl.DisplayAlerts = False
    wb.Save()
    print("wb.Save")
    xcl.Quit()
#unprotect_xlsx('C:\\Users\\guanz\\Desktop\\new_input\\input(1).xlsx')

def Remove_password_xlsx(filename, pw_str):
    import win32com.client
    xcl = win32com.client.Dispatch("Excel.Application")
    wb = xcl.Workbooks.Open(filename, False, False, None, pw_str)
    xcl.DisplayAlerts = True
    #wb.SaveAs(filename, None, '', '')
    #wb.Save()
    #wb.SaveAs(r"c:/users/jli/downloads/test.csv", FileFormat="6")
    print(xcl.parse(0))
    xcl.Quit()
#Remove_password_xlsx(r'C:/Users/jli/downloads/sample_insurer_2013.xlsx', '12345')

def saveasCSV(filename, output):
    #import win32com.client
    #xcl = win32com.client.Dispatch("Excel.Application")
    wb = xcl.Workbooks.Open(filename, False, False, None)
    xcl.DisplayAlerts = False
    #wb.SaveAs(filename, None, '', '')
    #wb.Save()
    wb.SaveAs(output, FileFormat="6")
    wb.Close(False)
    print("Produced ") + output
    #xcl.Quit()
#saveasCSV(r'C:/Users/jli/downloads/s1.xls', r'C:\Users\jli\downloads\s1.csv')

def saveasCSV1(filename):
    #import win32com.client
    #xcl = win32com.client.Dispatch("Excel.Application")
    filename = os.path.abspath(filename)
    print("saveasCSV1 in " + filename)
    wb = xcl.Workbooks.Open(filename, False, False, None)
    output = os.path.join(os.path.dirname(filename), os.path.splitext(os.path.basename(filename))[0]) + ".csv"
    wb.SaveAs(output, FileFormat="6")
    wb.Close(False)
    print("saveasCSV1 out " + output)
    #xcl.Quit()
#saveasCSV1(r'C:\Users\jli\downloads\s1.xls')

#non-encrypted: creates csv per worksheet
def saveasCSV2(filename):
    filename = os.path.abspath(filename)
    print("saveasCSV2 in " + filename)
    workbook = xlrd.open_workbook(filename) #Workbook is encrypted
    print(workbook.nsheets)
    print(workbook.sheet_names())
    all_worksheets = workbook.sheet_names()
    output = os.path.join(os.path.dirname(filename), os.path.splitext(os.path.basename(filename))[0])
    for worksheet_name in all_worksheets:
        worksheet = workbook.sheet_by_name(worksheet_name)
        if worksheet.nrows == 0: 
            continue
        csv_filename = output + '=' + worksheet_name + '.csv'
        fh = open(csv_filename, 'wb')
        csv_out = unicodecsv.writer(fh, encoding='utf-8')

        for row_number in range (worksheet.nrows):
            csv_out.writerow(worksheet.row_values(row_number))

        fh.close()
#saveasCSV2(r'C:\Users\jli\downloads\s1.xls')


def opensave(filename):
    print("opensave")
    print(filename)
    import win32com.client
    xcl = win32com.client.Dispatch("Excel.Application")
    wb = xcl.Workbooks.Open(filename, False, False, None)
    xcl.DisplayAlerts = True
    wb.SaveCopyAs(r'c:\users\jli\desktop\test.xls')
    wb.Close()
    #print(xcl.parse(0))
    xcl.Quit()
#opensave(r'C:/Users/jli/downloads/input(1001)2.xlsx')

In [ ]:


def parse_input(df14, df15, df16, xls):
    
    sheet0 = xls.parse(0)
    
    # 2014, 2015, and 2016 format
    if sheet0.iloc[2,0] == 'Company Name:': # should say "Company Name:"
        # hios id is 11C (9,1) in sheet 0
        # company name 4C (2,1) is in sheet 0
        # company state is 12C (10,1) in sheet 0
        # print(sheet0.iloc[9,1]) hios id
        # print(sheet0.iloc[2,1]) company name
        # print(sheet0.iloc[10,1]) company state 
        # print(sheet0.iloc[16,1]) reporting year
        reporting_year = sheet0.iloc[16,1]
        if reporting_year == 'No':
            reporting_year = sheet0.iloc[17,1]
        
        # individual member months is in section 1.7 59E (57,3) in sheet 1
        # small group member months is in section 1.7 59K (57,9) in sheet 1
        
        sheet1 = xls.parse(1)
        # print(sheet1.iloc[57,3]) individual member months
        # print(sheet1.iloc[57,9]) small group member months

        # reinsurance is in section 1.9 which is 15E in sheet 2
        # individual risk adjustment is in section 1.10 which is 16E in sheet 2
        # small group risk adjustment is in section 1.10 which is 16K in sheet 2
        sheet2 = xls.parse(2)
        # print(sheet2.iloc[13,3]) reinsurance
        # print(sheet2.iloc[14,3]) individual risk adjustment
        # print(sheet2.iloc[14,9]) small group risk adjustment

        if reporting_year == '2014':
            df14.loc[df14.shape[0]] = [sheet0.iloc[9,1], # hios id
                                   sheet0.iloc[2,1],  # company name
                                   sheet0.iloc[10,1], # company state
                                   sheet1.iloc[57,3], # individual member months
                                   sheet1.iloc[57,9], # small group member months
                                   sheet2.iloc[13,3], # reinsurance
                                   sheet2.iloc[14,3], # individual risk adjustment
                                   sheet2.iloc[14,9]] # small group risk adjustment
        elif reporting_year == '2015':
            df15.loc[df15.shape[0]] = [sheet0.iloc[9,1], # hios id
                                   sheet0.iloc[2,1],  # company name
                                   sheet0.iloc[10,1], # company state
                                   sheet1.iloc[57,3], # individual member months
                                   sheet1.iloc[57,9], # small group member months
                                   sheet2.iloc[13,3], # reinsurance
                                   sheet2.iloc[14,3], # individual risk adjustment
                                   sheet2.iloc[14,9]] # small group risk adjustment
        elif reporting_year == '2016':
            df16.loc[df16.shape[0]] = [sheet0.iloc[9,1], # hios id
                                   sheet0.iloc[2,1],  # company name
                                   sheet0.iloc[10,1], # company state
                                   sheet1.iloc[57,3], # individual member months
                                   sheet1.iloc[57,9], # small group member months
                                   sheet2.iloc[13,3], # reinsurance
                                   sheet2.iloc[14,3], # individual risk adjustment
                                   sheet2.iloc[14,9]] # small group risk adjustment
    # else: # 2011, 2012, 2013 format
        #print(sheet0.iloc[9,3]) # hios id
        #print(sheet0.iloc[4,2]) # company name
        #print(sheet0.iloc[8,7]) # company state
        #print(sheet0.iloc[79,8]) # individual member months
        #print(sheet0.iloc[79,13]) # small group member months
        # reinsurance, risk adjustment missing?
    
    #print("parse_input defined")

In [ ]:
# create the dataframe to run actual script

errorcount = 0

mydf14 = pd.DataFrame(columns=('HIOS_ID', 'COMPANY_NAME', 'STATE', 
                        'IND_MEMBER_MONTHS', 'SG_MEMBER_MONTHS',
                       'REINSURANCE', 'IND_RISK_ADJ', 'SG_RISK_ADJ'))
mydf15 = pd.DataFrame(columns=('HIOS_ID', 'COMPANY_NAME', 'STATE', 
                        'IND_MEMBER_MONTHS', 'SG_MEMBER_MONTHS',
                       'REINSURANCE', 'IND_RISK_ADJ', 'SG_RISK_ADJ'))
mydf16 = pd.DataFrame(columns=('HIOS_ID', 'COMPANY_NAME', 'STATE', 
                        'IND_MEMBER_MONTHS', 'SG_MEMBER_MONTHS',
                       'REINSURANCE', 'IND_RISK_ADJ', 'SG_RISK_ADJ'))

print("There were " + str(errorcount) + " encrypted files.")
print(mydf14)
print(mydf15)
print(mydf16)